Dependencies in this project:

```python
import requests
import re
import random
import jx_telebot as jx
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import pandas_datareader.data as web
import yfinance as yf
```

Incantations:

```python
%load_ext autoreload
%autoreload 2
```

# Set reference date

In [1]:
from datetime import datetime, timedelta

In [2]:
# set the reference date - default by referencing today()
date_t = datetime.today()
date = date_t.date()
date

datetime.date(2020, 7, 18)

# Gather necessary financial data

In [3]:
import jx_telebot as jx
%load_ext autoreload
%autoreload 2

In [4]:
# list of APAC indices we're interested in
apac_indices = [
    '^STI', 
    '^HSI', 
    '000300.SS', 
    '000001.SS', 
    '399006.SZ', 
    '^N225', 
    '^AXJO', 
    '^AORD'
]

In [4]:
# list of US/EU indices we're interested in
useu_indices = [
    '^GSPC',
    '^DJI',
    '^IXIC',
    '^RUT',
    '^STOXX',
    '^FTSE',
    '^STOXX50E',
    '^GDAXI',
    '^FCHI',
]

In [5]:
# list of other indicators we're interested in
othr_list = [
    '^VIX',
    'ES=F',
    'YM=F',
    'NQ=F',
    'GC=F',
    'CL=F',
    'SGD=X',
    '^IRX',
    '^FVX',
    '^TNX',
    '^TYX',
]

In [6]:
# list of equities we want data for
stocks_list_us = ['AMZN', 'FB', 'AAPL', 'GOOG']
stocks_list_sg = ['D05.SI', 'U11.SI', 'O39.SI']

## Capture data in our dataframes

In [8]:
apac_df = jx.get_prev_summary(apac_indices, date, series_type='apac')
apac_df

000300.SS doesn't have historical data. Retrieving values from local data dated 2020-07-16.
399006.SZ doesn't have historical data. Retrieving values from local data dated 2020-07-16.


,short_name,close_price,pct_chg
^STI,STI Index,2618.48,-0.001978
^HSI,HANG SENG INDEX,25089.17,0.004745
000300.SS,CSI 300 Index,4544.70,0.006299
000001.SS,SSE Composite Index,3214.13,0.001255
399006.SZ,CHINEXT PRICE INDEX,2662.40,0.006099
^N225,Nikkei 225,22696.42,-0.003247
^AXJO,S&P/ASX 200,6033.60,0.003776
^AORD,ALL ORDINARIES,6144.90,0.003577


In [7]:
#useu_df = jx.get_prev_summary(useu_indices, date, series_type='useu')
useu_df = jx.get_prev_summary(useu_indices, date - timedelta(1), series_type='useu')
useu_df

,short_name,close_price,pct_chg
^GSPC,S&P 500,3224.73,0.002849
^DJI,Dow Jones Industrial Average,26671.95,-0.002348
^IXIC,NASDAQ Composite,10503.19,0.002803
^RUT,Russell 2000,1473.32,0.003925
^STOXX,STXE 600 PR.EUR,372.71,0.001559
^FTSE,FTSE 100,6290.30,0.006335
^STOXX50E,ESTX 50 PR.EUR,3365.60,0.000074
^GDAXI,DAX PERFORMANCE-INDEX,12919.61,0.003467
^FCHI,CAC 40,5069.42,-0.003119


In [8]:
#othr_df = jx.get_prev_summary(othr_list, date, series_type='othr')
othr_df = jx.get_prev_summary(othr_list, date - timedelta(1), series_type='othr')
othr_df

,short_name,close_price,pct_chg
^VIX,CBOE Volatility Index,25.6800,-0.082857
ES=F,E-Mini S&P 500 Sep 20,3213.7500,0.003748
YM=F,Mini Dow Jones Indus.-$5 Sep 20,26521.0000,-0.003869
NQ=F,Nasdaq 100 Sep 20,10622.5000,0.005585
GC=F,Gold Aug 20,1812.1000,0.002767
CL=F,Crude Oil Aug 20,40.5700,-0.004661
SGD=X,USD/SGD,1.3919,0.001871
^IRX,13 WEEK TREASURY BILL,0.1050,0.019417
^FVX,Treasury Yield 5 Years,0.2820,0.025455
^TNX,CBOE Interest Rate 10 Year T No,0.6280,0.026144


In [9]:
stocks_df_us = jx.get_prev_summary(stocks_list_us, date - timedelta(1))
stocks_df_us

,short_name,close_price,pct_chg
AMZN,"Amazon.com, Inc.",2961.97,-0.012644
FB,"Facebook, Inc.",242.03,0.004566
AAPL,Apple Inc.,385.31,-0.002020
GOOG,Alphabet Inc.,1515.55,-0.001614


In [11]:
stocks_df_sg = jx.get_prev_summary(stocks_list_sg, date)
stocks_df_sg

,short_name,close_price,pct_chg
D05.SI,DBS,21.38,-0.010185
U11.SI,NaN,20.56,-0.004358
O39.SI,NaN,9.17,0.000000


## Create the working dataframes for the draft message

In [12]:
working_df_pm = apac_df.append([useu_df, othr_df, stocks_df_sg])
working_df_pm

,short_name,close_price,pct_chg
^STI,STI Index,2618.4800,-0.001978
^HSI,HANG SENG INDEX,25089.1700,0.004745
000300.SS,CSI 300 Index,4544.7000,0.006299
000001.SS,SSE Composite Index,3214.1300,0.001255
399006.SZ,CHINEXT PRICE INDEX,2662.4000,0.006099
^N225,Nikkei 225,22696.4200,-0.003247
^AXJO,S&P/ASX 200,6033.6000,0.003776
^AORD,ALL ORDINARIES,6144.9000,0.003577
^GSPC,S&P 500,NaN,NaN
^DJI,Dow Jones Industrial Average,NaN,NaN


In [10]:
working_df_am = useu_df.append([othr_df, stocks_df_us])
working_df_am

,short_name,close_price,pct_chg
^GSPC,S&P 500,3224.7300,0.002849
^DJI,Dow Jones Industrial Average,26671.9500,-0.002348
^IXIC,NASDAQ Composite,10503.1900,0.002803
^RUT,Russell 2000,1473.3200,0.003925
^STOXX,STXE 600 PR.EUR,372.7100,0.001559
^FTSE,FTSE 100,6290.3000,0.006335
^STOXX50E,ESTX 50 PR.EUR,3365.6000,0.000074
^GDAXI,DAX PERFORMANCE-INDEX,12919.6100,0.003467
^FCHI,CAC 40,5069.4200,-0.003119
^VIX,CBOE Volatility Index,25.6800,-0.082857


## Save data to csv file

In [11]:
#jx.update_csv(apac_df, date, path='apac_timeseries.csv')
jx.update_csv(useu_df, date - timedelta(1), path='useu_timeseries.csv')
jx.update_csv(othr_df, date - timedelta(1), path='othr_timeseries.csv')

In [ ]:
# jx.undo_csv('othr_timeseries.csv')
# jx.undo_csv('useu_timeseries.csv')
jx.undo_csv('apac_timeseries.csv')

## To preview the stored csv file thus far

In [14]:
jx.return_csv('apac')

,^STI,^HSI,000300.SS,000001.SS,399006.SZ,^N225,^AXJO,^AORD
date,,,,,,,,
2020-03-07,2652.94,25373.12,4419.60,3152.81,2462.56,22306.48,6057.9,6163.7
2020-06-07,2689.61,26339.16,4670.09,3332.88,2529.49,22714.44,6014.6,6125.9
2020-07-07,2661.42,25975.66,4698.13,3345.34,2591.26,22614.69,6012.9,6126.7
2020-08-07,2669.49,26129.18,4774.00,3403.44,2651.97,22438.65,5920.3,6034.3
2020-09-07,2652.65,26210.16,4840.77,3450.59,2757.65,22529.29,5955.5,6074.9
2020-10-07,NaN,25727.41,4753.13,3383.32,2778.46,22290.81,5919.2,6036.3
2020-07-13,2631.08,25772.12,4852.96,3443.29,2889.43,22784.74,5977.5,6089.3
2020-07-14,2620.19,25477.89,4806.69,3414.62,2858.67,22587.01,5941.1,6045.5
2020-07-15,2648.90,25481.58,4744.47,3361.30,2813.06,22945.50,6052.9,6160.4


In [12]:
jx.return_csv('useu')

,^GSPC,^DJI,^IXIC,^RUT,^STOXX,^FTSE,^STOXX50E,^GDAXI,^FCHI
date,,,,,,,,,
2020-07-02,3130.01,25827.36,10207.63,1431.86,368.29,6240.40,3320.09,12608.46,5049.38
2020-07-03,NaN,NaN,NaN,NaN,365.43,6157.30,3294.38,12528.18,5007.14
2020-07-06,3179.72,26287.03,10433.65,1442.88,371.21,6285.90,3350.03,12733.45,5081.51
2020-07-07,3145.32,25890.18,10343.89,1416.00,368.96,6189.90,3321.56,12616.80,5043.73
2020-07-08,3169.94,26067.28,10492.50,1427.40,366.48,6156.16,3286.09,12494.81,4981.13
2020-07-09,3152.05,25706.09,10547.75,1398.92,363.64,6049.60,3261.17,12489.46,4921.01
2020-07-10,3185.04,26075.30,10617.44,1422.68,366.83,6095.40,3296.22,12633.71,4970.48
2020-07-13,3155.22,26085.80,10390.84,1403.57,370.50,6176.19,3350.00,12799.97,5056.23
2020-07-14,3197.52,26642.59,10488.58,1428.26,367.40,6179.75,3321.39,12697.36,5007.46


In [13]:
jx.return_csv('othr')

,^VIX,ES=F,YM=F,NQ=F,GC=F,CL=F,SGD=X,^IRX,^FVX,^TNX,^TYX
date,,,,,,,,,,,
2020-07-02,27.68,3129.50,25762.0,10354.75,1787.0,40.32,1.3936,0.133,0.294,0.669,1.429
2020-07-03,NaN,3115.50,25623.0,10326.00,1787.3,40.32,1.3951,NaN,NaN,NaN,NaN
2020-07-06,27.94,3177.00,26224.0,10640.75,1794.7,40.70,1.3938,0.128,0.307,0.684,1.443
2020-07-07,29.43,3141.25,25825.0,10553.00,1807.6,40.41,1.3919,0.140,0.293,0.650,1.390
2020-07-08,28.08,3165.75,25992.0,10670.00,1820.0,40.92,1.3954,0.135,0.291,0.653,1.392
2020-07-09,29.26,3152.25,25670.0,10752.00,1808.8,39.65,1.3908,0.123,0.274,0.605,1.308
2020-07-10,27.29,3179.50,25980.0,10843.75,1801.8,40.62,1.3918,0.120,0.298,0.633,1.326
2020-07-13,32.19,3154.50,26061.0,10650.50,1803.7,39.55,1.3909,0.125,0.298,0.640,1.337
2020-07-14,29.52,3210.75,26785.0,10716.00,1812.1,40.73,1.3898,0.133,0.282,0.615,1.302


# Prep the output text

PM edition

In [15]:
# recall: date_t = datetime.today() - timedelta(whatever)
df = working_df_pm
out_path = 'pm_msg {}.txt'.format(date.strftime(format='%Y%m%d'))
author = 'JX'
text = jx.message_draft_pm(df=df, out_path=out_path, author=author, date=date_t)
print(text)

__Friday market roundup - 17 Jul 2020 (PM)__
By JX

*Singapore* 
The Straits Times Index edged down 0.2% to 2618.48.

The banking trio were mixed; DBS shedded 1.02% to 21.38, OCBC remained flat at 9.17, and UOB declined 0.44% to 20.56.

*Hong Kong*
The Hang Seng Index added 0.47% to 25089.17.

*China*
The CSI 300 rose 0.63% to 4544.7, while the SSE Composite index edged up 0.13% to 3214.13. The tech-heavy Chinext Price Index gained 0.61% to 2662.4.

*Japan*
The Nikkei 225 retreated by 0.32% to 22696.42.

*Australia*
The ASX 200 advanced 0.38% to 6033.6, while the broader All Ordinaries Index was 0.36% higher at 6144.9.

*UK*
The FTSE 100 rose 0.37% to 6274.11 as of 06:08 PM. 

*Europe*
As of 06:08 PM, the pan-European Stoxx 600 inched up 0.09% to 372.46, while the blue-chip Stoxx 50 index inched up 0.12% to 3369.53. Elsewhere, the CAC 40 was 0.24% lower at 5073.12 in Paris, while the DAX climbed 0.35% to 12920.6 in Frankfurt. 

*Others*
As of 06:08 PM:
E-Mini S&P 500 Sep 20 futures edg

AM edition

In [14]:
# recall: date_t = datetime.today() - timedelta(whatever)
df = working_df_am
out_path = 'am_msg {}.txt'.format(date.strftime(format='%Y%m%d'))
author = 'JX'
text = jx.message_draft_am(df=df, out_path=out_path, author=author, date=date_t)
print(text)

__Saturday market roundup - 18 Jul 2020 (AM)__
By JX

*US*
The S&P 500 gained 0.28% to 3224.73, the Dow Jones Industrial Average was 0.23% lower at 26671.95, and the NASDAQ Composite rose 0.28% to 10503.19 on Friday. 

Among the tech heavyweights, Facebook Inc climbed 0.46% to 242.03, Amazon.com Inc slid 1.26% to 2961.97, Apple Inc was 0.2% lower at 385.31, and Alphabet Inc inched down 0.16% to 1515.55. 

*UK*
The FTSE 100 added 0.63% to 6290.3 on Friday. 

*Europe*
On Friday the pan-European Stoxx 600 inched up 0.16% to 372.71, while the blue-chip Stoxx 50 index edged up 0.01% to 3365.6. The CAC 40 dropped 0.31% to 5069.42 in Paris, while the DAX climbed 0.35% to 12919.61 in Frankfurt. 

*Others*
As of 10:41 AM:
Gold Aug 20 futures added 0.28% to 1812.1.
Crude Oil Aug 20 contracts shedded 0.47% to 40.57
Ten-year treasury yields surged 2.61% to 0.63.
USD/SGD inched up 0.19% to 1.39.


# Mess around with the Telegram bot

In [15]:
bot = jx.telegram_bot()

In [ ]:
bot.send_msg_file(out_path)